https://pytorch.org/tutorials/beginner/basics/intro.html

PyTorchは2つのデータセットがある。  
torch.utils.data.DataLoader：Datasetのラッパー  
torch.utils.data.Dataset

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Pytorchにはドメインごとにライブラリが存在する。TorchText, TorchVision, and TorchAudio.

FashionMNISTを使用する。

In [2]:
# download train datasets
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# download test datasets
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models
PyTorchでニューラルネットワークを定義するためには、nn.Moduleを継承したクラスを作成する。レイヤーは```__init__```関数で定義する。データが伝播する方式は```forward```関数で定義する。

In [15]:
# Get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


optimizing the model parameters

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [26]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, corrent = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{corrent:>5d}/{size:>5d}]")

In [27]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312848 [   64/60000]
loss: 2.297966 [  704/60000]
loss: 2.298067 [ 1344/60000]
loss: 2.301866 [ 1984/60000]
loss: 2.305833 [ 2624/60000]
loss: 2.303755 [ 3264/60000]
loss: 2.292912 [ 3904/60000]
loss: 2.287566 [ 4544/60000]
loss: 2.299021 [ 5184/60000]
loss: 2.299621 [ 5824/60000]
loss: 2.297798 [ 6464/60000]
loss: 2.289417 [ 7104/60000]
loss: 2.284974 [ 7744/60000]
loss: 2.284792 [ 8384/60000]
loss: 2.286625 [ 9024/60000]
loss: 2.279461 [ 9664/60000]
loss: 2.287282 [10304/60000]
loss: 2.283288 [10944/60000]
loss: 2.277294 [11584/60000]
loss: 2.277409 [12224/60000]
loss: 2.275964 [12864/60000]
loss: 2.280407 [13504/60000]
loss: 2.272880 [14144/60000]
loss: 2.267057 [14784/60000]
loss: 2.277332 [15424/60000]
loss: 2.278931 [16064/60000]
loss: 2.267663 [16704/60000]
loss: 2.268663 [17344/60000]
loss: 2.273103 [17984/60000]
loss: 2.273017 [18624/60000]
loss: 2.268409 [19264/60000]
loss: 2.260637 [19904/60000]
loss: 2.267769 [20544/60000]
los

In [29]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [30]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[-0.0146, -0.0073,  0.0150,  ...,  0.0017, -0.0130,  0.0177],
                      [ 0.0314, -0.0279, -0.0003,  ..., -0.0083,  0.0152, -0.0196],
                      [ 0.0186,  0.0092, -0.0058,  ...,  0.0276, -0.0184,  0.0029],
                      ...,
                      [ 0.0122, -0.0065,  0.0026,  ...,  0.0119, -0.0122, -0.0115],
                      [ 0.0282,  0.0267,  0.0270,  ..., -0.0250,  0.0276,  0.0245],
                      [-0.0321, -0.0226,  0.0324,  ..., -0.0057, -0.0308, -0.0293]],
                     device='mps:0')),
             ('linear_relu_stack.0.bias',
              tensor([ 0.0156,  0.0418,  0.0314,  0.0044,  0.0359, -0.0142,  0.0208, -0.0014,
                      -0.0018, -0.0336, -0.0100, -0.0079,  0.0127,  0.0019,  0.0253, -0.0168,
                       0.0368,  0.0111,  0.0202, -0.0266, -0.0197,  0.0079, -0.0307,  0.0315,
                      -0.0233,  0.0217,  0.0121,  0.0423, -0.

In [31]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

This model can now be used to make predictions.

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
